In [1]:
from __future__ import print_function
from six.moves import cPickle as pickle
import tensorflow as tf

In [2]:
f=open('Data.pickle','rb')
Data=open('Data.pickle','rb')
train_dataset=pickle.load(Data)['train_dataset']
Data.seek(0)
train_labels=pickle.load(Data)['train_labels']
Data.seek(0)
test_dataset=pickle.load(Data)['test_dataset']
Data.seek(0)
test_labels=pickle.load(Data)['test_labels']
Data.close()

In [3]:
print(train_dataset.shape)

(50000, 28, 28)


In [4]:
train_dataset=train_dataset.reshape(train_dataset.shape[0],-1).T
train_labels=train_labels.reshape(train_labels.shape[0],-1).T
test_dataset=train_dataset.reshape(test_dataset.shape[0],-1).T
test_labels=train_labels.reshape(test_labels.shape[0],-1).T

In [5]:
def One_hot_encoder(labels,C):
    C=tf.constant(C,dtype=tf.int32,name='C')
    encoded=tf.one_hot(labels,C)
    with tf.Session() as sess:
        return sess.run(encoded)

In [6]:
Train_labels=One_hot_encoder(train_labels,10)
#rint(Train_labels.shape)
Train_labels=Train_labels[0]
print(Train_labels.shape)
#rint(train_labels[0])
#rint(Train_labels[0][0])
print(train_dataset.shape)

(50000, 10)
(784, 50000)


In [7]:
total_layers=15
Layer_Units={1:38,2:38,3:38,4:38,5:38,6:38,7:38,8:38,9:38,10:38,11:38,12:38,13:38,14:38}
Layer_Units[total_layers]=10
cache={}
for i in range(1,total_layers+1):
    cache['Z'+str(i)]=0
    cache['A'+str(i)]=0
parameters={}
count=1
for i in Layer_Units:
    num=i
    weight="W"+str(num)
    bias='b'+str(num)
    if i==1:
        parameters[weight]=tf.get_variable(weight,[Layer_Units[i],784],initializer=tf.contrib.layers.xavier_initializer(seed=1))
        parameters[bias] = tf.get_variable(bias, [Layer_Units[i], 1], initializer = tf.zeros_initializer())
    else:
        parameters[weight]=tf.get_variable(weight,[Layer_Units[i],Layer_Units[i-1]],initializer=tf.contrib.layers.xavier_initializer(seed=1)) 
        parameters[bias] = tf.get_variable(bias, [Layer_Units[i], 1], initializer = tf.zeros_initializer())

In [8]:
for i in Layer_Units:
        num=i
        if i==1:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],train_dataset),parameters['b'+str(num)])
            cache['A'+str(i)]=tf.nn.relu(cache['Z'+str(i)])   
        elif i==total_layers:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],cache['A'+str(i-1)]),parameters['b'+str(num)])
        else:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],cache['A'+str(i-1)]),parameters['b'+str(num)])
            cache['A'+str(i)]=tf.nn.relu(cache['Z'+str(i)])

In [10]:
logits = tf.transpose(cache['Z'+str(total_layers)])
labels = Train_labels
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels =labels))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
count=1
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print('Number','Loss','Accuracy')
    for i in range(10000):
        session.run(train_op)
        loss, acc = session.run([cost, accuracy])
        print(count,':',loss,':',acc)
        count=count+1   

Number Loss Accuracy
1 : 2.30217 : 0.09874
2 : 2.29995 : 0.09874
3 : 2.29763 : 0.11658
4 : 2.29429 : 0.12194
5 : 2.28954 : 0.11118
6 : 2.28387 : 0.10922
7 : 2.27545 : 0.10956
8 : 2.26211 : 0.11878
9 : 2.24571 : 0.16836
10 : 2.22761 : 0.17578
11 : 2.20744 : 0.17324
12 : 2.18835 : 0.18418
13 : 2.16773 : 0.19244
14 : 2.15114 : 0.1926
15 : 2.13813 : 0.19468
16 : 2.13011 : 0.19768
17 : 2.12194 : 0.20122
18 : 2.116 : 0.1978
19 : 2.10829 : 0.1965
20 : 2.10093 : 0.18698
21 : 2.09219 : 0.18484
22 : 2.0817 : 0.18872
23 : 2.07439 : 0.19496
24 : 2.06787 : 0.19984
25 : 2.0585 : 0.2033
26 : 2.05669 : 0.20564
27 : 2.04957 : 0.22162
28 : 2.04108 : 0.22632
29 : 2.03175 : 0.22166
30 : 2.0204 : 0.22528
31 : 2.01284 : 0.22854
32 : 2.00224 : 0.23094
33 : 1.99323 : 0.22628
34 : 1.97985 : 0.2381
35 : 1.96926 : 0.28244
36 : 1.95505 : 0.29252
37 : 1.94096 : 0.29604
38 : 1.92545 : 0.30272
39 : 1.90716 : 0.30772
40 : 1.88987 : 0.31198
41 : 1.87055 : 0.31176
42 : 1.84691 : 0.31406
43 : 1.82309 : 0.31512
44 : 1.79

341 : 0.43536 : 0.84836
342 : 0.432614 : 0.84922
343 : 0.430748 : 0.85078
344 : 0.431408 : 0.85024
345 : 0.432206 : 0.84926
346 : 0.431065 : 0.85044
347 : 0.429752 : 0.85064
348 : 0.430029 : 0.85062
349 : 0.430751 : 0.84968
350 : 0.429793 : 0.8503
351 : 0.42794 : 0.85136
352 : 0.427105 : 0.85176
353 : 0.427693 : 0.85164
354 : 0.42805 : 0.85104
355 : 0.427337 : 0.8512
356 : 0.426169 : 0.85164
357 : 0.425862 : 0.85214
358 : 0.426648 : 0.85118
359 : 0.427651 : 0.85104
360 : 0.428481 : 0.85002
361 : 0.43025 : 0.84942
362 : 0.434792 : 0.8468
363 : 0.444434 : 0.84328
364 : 0.460586 : 0.83712
365 : 0.480566 : 0.83038
366 : 0.495131 : 0.82536
367 : 0.500161 : 0.8249
368 : 0.456007 : 0.83888
369 : 0.449648 : 0.84138
370 : 0.477124 : 0.8325
371 : 0.439501 : 0.84496
372 : 0.462609 : 0.83768
373 : 0.446648 : 0.8425
374 : 0.44895 : 0.842
375 : 0.446528 : 0.84258
376 : 0.434012 : 0.84724
377 : 0.444956 : 0.84386
378 : 0.427646 : 0.85036
379 : 0.44051 : 0.845
380 : 0.425337 : 0.85172
381 : 0.436071 :

675 : 0.392367 : 0.86162
676 : 0.392097 : 0.86164
677 : 0.391834 : 0.86168
678 : 0.391567 : 0.86178
679 : 0.391308 : 0.8619
680 : 0.391049 : 0.8621
681 : 0.390794 : 0.86216
682 : 0.390537 : 0.86224
683 : 0.39029 : 0.86228
684 : 0.390047 : 0.86244
685 : 0.389806 : 0.86246
686 : 0.389564 : 0.86254
687 : 0.389333 : 0.86254
688 : 0.389101 : 0.8626
689 : 0.388871 : 0.86258
690 : 0.38865 : 0.86262
691 : 0.388429 : 0.86276
692 : 0.388216 : 0.86288
693 : 0.388004 : 0.86292
694 : 0.387793 : 0.863
695 : 0.387582 : 0.86308
696 : 0.387376 : 0.86324
697 : 0.387174 : 0.86334
698 : 0.386973 : 0.86344
699 : 0.386775 : 0.86348
700 : 0.386577 : 0.86348
701 : 0.386389 : 0.86348
702 : 0.386194 : 0.8636
703 : 0.386005 : 0.86362
704 : 0.38582 : 0.86364
705 : 0.385635 : 0.8637
706 : 0.385448 : 0.86378
707 : 0.385267 : 0.86378
708 : 0.385084 : 0.86386
709 : 0.384907 : 0.86398
710 : 0.38473 : 0.86402
711 : 0.384553 : 0.86414
712 : 0.384381 : 0.86416
713 : 0.384207 : 0.86424
714 : 0.384042 : 0.86434
715 : 0.383

1007 : 0.355754 : 0.87246
1008 : 0.355679 : 0.8725
1009 : 0.355637 : 0.87256
1010 : 0.355587 : 0.87254
1011 : 0.355505 : 0.87256
1012 : 0.355427 : 0.87262
1013 : 0.355359 : 0.87264
1014 : 0.355262 : 0.87266
1015 : 0.355182 : 0.87264
1016 : 0.355118 : 0.87262
1017 : 0.355066 : 0.87272
1018 : 0.355028 : 0.87268
1019 : 0.35501 : 0.87274
1020 : 0.354929 : 0.87278
1021 : 0.354839 : 0.8728
1022 : 0.354729 : 0.87278
1023 : 0.354636 : 0.87284
1024 : 0.354563 : 0.87284
1025 : 0.35449 : 0.87288
1026 : 0.354418 : 0.87288
1027 : 0.354349 : 0.8729
1028 : 0.354299 : 0.87296
1029 : 0.354254 : 0.8729
1030 : 0.354239 : 0.87294
1031 : 0.35419 : 0.87294
1032 : 0.354145 : 0.8729
1033 : 0.35403 : 0.87298
1034 : 0.353926 : 0.873
1035 : 0.353836 : 0.87302
1036 : 0.353751 : 0.873
1037 : 0.353667 : 0.87302
1038 : 0.353613 : 0.87306
1039 : 0.353572 : 0.8731
1040 : 0.353535 : 0.87304
1041 : 0.353504 : 0.87312
1042 : 0.35345 : 0.87298
1043 : 0.353352 : 0.87312
1044 : 0.353239 : 0.87308
1045 : 0.353145 : 0.87316
1

1327 : 0.38312 : 0.86374
1328 : 0.382391 : 0.8638
1329 : 0.381702 : 0.86394
1330 : 0.381048 : 0.86422
1331 : 0.380395 : 0.86436
1332 : 0.379761 : 0.86454
1333 : 0.379155 : 0.86476
1334 : 0.378559 : 0.86502
1335 : 0.377963 : 0.86526
1336 : 0.37738 : 0.86536
1337 : 0.376804 : 0.86562
1338 : 0.376254 : 0.86584
1339 : 0.375705 : 0.86602
1340 : 0.37515 : 0.86616
1341 : 0.374619 : 0.86622
1342 : 0.374097 : 0.86632
1343 : 0.373587 : 0.86652
1344 : 0.373088 : 0.86662
1345 : 0.372587 : 0.86672
1346 : 0.372099 : 0.8668
1347 : 0.371627 : 0.8669
1348 : 0.37115 : 0.86698
1349 : 0.370698 : 0.8671
1350 : 0.370245 : 0.86722
1351 : 0.369802 : 0.86744
1352 : 0.36937 : 0.86756
1353 : 0.368942 : 0.86774
1354 : 0.368533 : 0.86784
1355 : 0.368125 : 0.86798
1356 : 0.367726 : 0.86828
1357 : 0.36734 : 0.86834
1358 : 0.366957 : 0.86842
1359 : 0.366588 : 0.86858
1360 : 0.36622 : 0.86858
1361 : 0.365859 : 0.86884
1362 : 0.365501 : 0.869
1363 : 0.365154 : 0.86916
1364 : 0.364815 : 0.86932
1365 : 0.364481 : 0.86942

...

8686 : 0.326164 : 0.87996
8687 : 0.32611 : 0.87998
8688 : 0.32608 : 0.88
8689 : 0.326108 : 0.87996
8690 : 0.326085 : 0.87996
8691 : 0.326088 : 0.88002
8692 : 0.326041 : 0.87996
8693 : 0.326048 : 0.87994
8694 : 0.326023 : 0.87998
8695 : 0.325984 : 0.88
8696 : 0.325991 : 0.87998
8697 : 0.325961 : 0.88
8698 : 0.325933 : 0.88006
8699 : 0.325914 : 0.88004
8700 : 0.325932 : 0.88002
8701 : 0.325927 : 0.88002
8702 : 0.32588 : 0.88004
8703 : 0.325911 : 0.87996
8704 : 0.325858 : 0.88002
8705 : 0.325903 : 0.88004
8706 : 0.325812 : 0.88004
8707 : 0.325873 : 0.87998
8708 : 0.325778 : 0.88004
8709 : 0.325827 : 0.88002
8710 : 0.325768 : 0.88004
8711 : 0.325819 : 0.88002
8712 : 0.325733 : 0.88006
8713 : 0.325791 : 0.87996
8714 : 0.3257 : 0.88006
8715 : 0.325723 : 0.88008
8716 : 0.325685 : 0.88006
8717 : 0.325687 : 0.88004
8718 : 0.325638 : 0.88008
8719 : 0.325641 : 0.88006
8720 : 0.325631 : 0.88004
8721 : 0.325596 : 0.88006
8722 : 0.325603 : 0.88006
8723 : 0.325578 : 0.88008
8724 : 0.325569 : 0.88006


9005 : 0.323074 : 0.8805
9006 : 0.323096 : 0.88056
9007 : 0.323049 : 0.88052
9008 : 0.323093 : 0.88056
9009 : 0.323314 : 0.8805
9010 : 0.323309 : 0.8805
9011 : 0.323084 : 0.88048
9012 : 0.323249 : 0.88048
9013 : 0.323067 : 0.88052
9014 : 0.323196 : 0.88046
9015 : 0.323147 : 0.8805
9016 : 0.323114 : 0.88058
9017 : 0.323134 : 0.8805
9018 : 0.323093 : 0.8805
9019 : 0.323062 : 0.88052
9020 : 0.323051 : 0.88052
9021 : 0.323034 : 0.88052
9022 : 0.323019 : 0.88058
9023 : 0.323107 : 0.88058
9024 : 0.323066 : 0.8805
9025 : 0.322986 : 0.88052
9026 : 0.323056 : 0.88054
9027 : 0.322963 : 0.88052
9028 : 0.323027 : 0.88054
9029 : 0.322934 : 0.88058
9030 : 0.322978 : 0.88056
9031 : 0.322927 : 0.88056
9032 : 0.322941 : 0.88056
9033 : 0.32293 : 0.88056
9034 : 0.32291 : 0.88052
9035 : 0.322907 : 0.88052
9036 : 0.322889 : 0.88054
9037 : 0.322875 : 0.88058
9038 : 0.322871 : 0.88062
9039 : 0.322864 : 0.8806
9040 : 0.322851 : 0.88058
9041 : 0.322836 : 0.88056
9042 : 0.322905 : 0.8806
9043 : 0.323072 : 0.880

9324 : 0.321193 : 0.88092
9325 : 0.321204 : 0.88094
9326 : 0.321178 : 0.88094
9327 : 0.321179 : 0.88096
9328 : 0.321162 : 0.88096
9329 : 0.321171 : 0.88094
9330 : 0.321162 : 0.88096
9331 : 0.321152 : 0.88096
9332 : 0.321166 : 0.88092
9333 : 0.321167 : 0.88094
9334 : 0.321151 : 0.88094
9335 : 0.321134 : 0.88096
9336 : 0.321138 : 0.88096
9337 : 0.321125 : 0.88096
9338 : 0.321118 : 0.88096
9339 : 0.321119 : 0.88096
9340 : 0.321107 : 0.88096
9341 : 0.321127 : 0.88094
9342 : 0.321107 : 0.88096
9343 : 0.321117 : 0.88096
9344 : 0.321101 : 0.88096
9345 : 0.321096 : 0.88096
9346 : 0.321103 : 0.88096
9347 : 0.321085 : 0.88096
9348 : 0.321092 : 0.88096
9349 : 0.321078 : 0.88096
9350 : 0.321075 : 0.88096
9351 : 0.32107 : 0.88096
9352 : 0.321075 : 0.88096
9353 : 0.321093 : 0.88092
9354 : 0.321073 : 0.88096
9355 : 0.321062 : 0.88096
9356 : 0.321045 : 0.88096
9357 : 0.321042 : 0.88096
9358 : 0.321048 : 0.88096
9359 : 0.321046 : 0.88096
9360 : 0.321039 : 0.88098
9361 : 0.321027 : 0.88098
9362 : 0.3210

9646 : 0.969619 : 0.6901
9647 : 0.943248 : 0.70228
9648 : 0.921226 : 0.7116
9649 : 0.906274 : 0.71652
9650 : 0.891335 : 0.72304
9651 : 0.872662 : 0.73052
9652 : 0.855672 : 0.73474
9653 : 0.842938 : 0.7358
9654 : 0.831267 : 0.73734
9655 : 0.817061 : 0.7414
9656 : 0.804273 : 0.74628
9657 : 0.793354 : 0.74932
9658 : 0.78226 : 0.75222
9659 : 0.771212 : 0.756
9660 : 0.761386 : 0.75876
9661 : 0.751766 : 0.76102
9662 : 0.742464 : 0.76368
9663 : 0.734987 : 0.76536
9664 : 0.727645 : 0.76758
9665 : 0.719896 : 0.76966
9666 : 0.713088 : 0.77264
9667 : 0.707834 : 0.7735
9668 : 0.702327 : 0.77486
9669 : 0.696671 : 0.77626
9670 : 0.691889 : 0.77714
9671 : 0.687081 : 0.77848
9672 : 0.682027 : 0.77974
9673 : 0.677454 : 0.781
9674 : 0.673393 : 0.78244
9675 : 0.669156 : 0.78344
9676 : 0.664934 : 0.78448
9677 : 0.661149 : 0.78556
9678 : 0.657466 : 0.78676
9679 : 0.653665 : 0.78794
9680 : 0.650181 : 0.7887
9681 : 0.646935 : 0.78924
9682 : 0.643766 : 0.79012
9683 : 0.640474 : 0.79034
9684 : 0.637341 : 0.791

9965 : 0.463987 : 0.83968
9966 : 0.463848 : 0.83956
9967 : 0.463673 : 0.83964
9968 : 0.463495 : 0.83968
9969 : 0.463226 : 0.83974
9970 : 0.463055 : 0.83996
9971 : 0.462762 : 0.8399
9972 : 0.462613 : 0.8399
9973 : 0.462465 : 0.84002
9974 : 0.462374 : 0.83984
9975 : 0.46226 : 0.84026
9976 : 0.461894 : 0.83996
9977 : 0.461634 : 0.8403
9978 : 0.461491 : 0.84012
9979 : 0.461264 : 0.84018
9980 : 0.461033 : 0.84056
9981 : 0.460904 : 0.8404
9982 : 0.460796 : 0.84058
9983 : 0.460618 : 0.84032
9984 : 0.46045 : 0.84068
9985 : 0.460297 : 0.84056
9986 : 0.460207 : 0.84082
9987 : 0.459922 : 0.84068
9988 : 0.459615 : 0.84074
9989 : 0.459342 : 0.84082
9990 : 0.459274 : 0.84066
9991 : 0.459292 : 0.84088
9992 : 0.459309 : 0.8406
9993 : 0.459008 : 0.84102
9994 : 0.458519 : 0.84096
9995 : 0.458361 : 0.84094
9996 : 0.458449 : 0.841
9997 : 0.458328 : 0.84086
9998 : 0.458093 : 0.84132
9999 : 0.457674 : 0.84124
10000 : 0.457488 : 0.8411
